# Multi-modal Data Processing with Data-Juicer

This notebook explores Data-Juicer's approach to multi-modal data processing, with a focus on unified data format and format conversion capabilities. Multi-modal data involves additional considerations around file path management and cross-modal alignment compared to text-only datasets.

## In This Notebook

1. Data-Juicer's Unified Multi-modal Format - The foundation of multi-modal processing
2. Format Conversion Tools - Converting between popular multi-modal formats
3. Data Path Management - Handling file paths and data migration
4. Complete Processing Workflow - End-to-end multi-modal data processing
5. Best Practices and Considerations - Tips for real-world applications

## Setup

First, let's set up our environment and prepare some sample data for demonstration.

In [1]:
import json
import os

# Create directories for our examples
os.makedirs('data/multimodal_examples', exist_ok=True)
os.makedirs('data/converted_data', exist_ok=True)
os.makedirs('configs', exist_ok=True)

print("Environment setup complete!")

Environment setup complete!


## Data-Juicer's Unified Multi-modal Format

Data-Juicer introduces a novel intermediate text-based interleaved data format that unifies different modalities. This format is the key to Data-Juicer's powerful multi-modal processing capabilities.

In [2]:
# Example of Data-Juicer multi-modal format
dj_format_example = {
  "text": "<__dj__image> Antarctica is Earth's southernmost and least-populated continent. <|__dj__eoc|> "
          "<__dj__video> <__dj__audio> Situated almost entirely south of the Antarctic Circle and surrounded by the "
          "Southern Ocean (also known as the Antarctic Ocean), it contains the geographic South Pole. <|__dj__eoc|> "
          "Antarctica is the fifth-largest continent, being about 40% larger than Europe, "
          "and has an area of 14,200,000 km2 (5,500,000 sq mi). <__dj__image> <__dj__image> <|__dj__eoc|> "
          "Most of Antarctica is covered by the Antarctic ice sheet, "
          "with an average thickness of 1.9 km (1.2 mi). <|__dj__eoc|>",
  "images": [
    "path/to/the/image/of/antarctica_snowfield",
    "path/to/the/image/of/antarctica_map",
    "path/to/the/image/of/europe_map"
  ],
  "audios": [
    "path/to/the/audio/of/sound_of_waves_in_Antarctic_Ocean"
  ],
  "videos": [
    "path/to/the/video/of/remote_sensing_view_of_antarctica"
  ],
  "meta": {
    "src": "customized",
    "version": "0.1",
    "author": "example"
  },
  "stats": {
    "lang": "en",
    "image_widths": [224, 336, 512]
  }
}

# Key Points:
# • 4 chunks separated by <|__dj__eoc|>
# • 3 images, 1 audio, 1 video referenced by special tokens
# • Token order matches file list order
# • Each chunk is semantically coherent

## Format Conversion Tools

Data-Juicer provides comprehensive tools to convert between popular multi-modal dataset formats. This is crucial for:
- **Data Integration**: Combining datasets from different sources
- **Format Standardization**: Using a unified format for processing
- **Ecosystem Compatibility**: Converting back to target formats for downstream use

### Supported Formats:

| Format | Type | Description | Use Case |
|--------|------|-------------|----------|
| **LLaVA-like** | Image-Text | Visual instruction tuning format | VLM training |
| **MMC4-like** | Image-Text | Multimodal C4 document format | Large-scale pretraining |
| **WavCaps-like** | Audio-Text | Audio captioning format | Audio understanding |
| **Video-ChatGPT-like** | Video-Text | Video QA format | Video understanding |
| **Youku-mPLUG-like** | Video-Text | Chinese video-text format | Multilingual video tasks |
| **InternVid-like** | Video-Text | Video clip captioning | Video-text alignment |

In [3]:
# Create sample datasets in different formats for demonstration

# LLaVA-like format example
llava_sample = [
    {
        "id": "sample_1",
        "image": "../../../tests/ops/data/img1.png",
        "conversations": [
            {"from": "human", "value": "<image>\nWhat do you see in this image?"},
            {"from": "gpt", "value": "I can see a comfortable bed."}
        ]
    },
    {
        "id": "sample_2", 
        "image": "../../../tests/ops/data/img3.jpg",
        "conversations": [
            {"from": "human", "value": "<image>\nDescribe the weather in this photo."},
            {"from": "gpt", "value": "The weather in the photo is distinctly rainy."}
        ]
    }
]

# Save LLaVA sample
with open('data/multimodal_examples/llava_sample.json', 'w') as f:
    json.dump(llava_sample, f, indent=2)

print("Sample datasets created:")
print("LLaVA format: data/multimodal_examples/llava_sample.json")

Sample datasets created:
LLaVA format: data/multimodal_examples/llava_sample.json


### Format Conversion in Action

Let's demonstrate how to convert different formats to Data-Juicer format and back:

In [4]:
# Expected Data-Juicer Format (from LLaVA):
# {
#     "text": "<__dj__image> What do you see in this image? I can see a beautiful landscape with mountains and a lake.",
#     "images": ["path/to/image1.jpg"],
#     "meta": {"id": "sample_1", "src": "llava"}
# }
# LLaVA to Data-Juicer conversion
print("Converting LLaVA format to Data-Juicer format...")
!python ../tools/fmt_conversion/multimodal/source_format_to_data_juicer_format/llava_to_dj.py data/multimodal_examples/llava_sample.json data/converted_data/llava_to_dj.jsonl

# Data-Juicer back to LLaVA conversion
print("\nConverting Data-Juicer format back to LLaVA format...")
!python ../tools/fmt_conversion/multimodal/data_juicer_format_to_target_format/dj_to_llava.py data/converted_data/llava_to_dj.jsonl data/converted_data/dj_back_to_llava.json


print("Looking at the converted Data-Juicer format:")
with open("data/converted_data/llava_to_dj.jsonl", "r") as f:
    lines = f.readlines()
    print("llava_to_dj:")
    for line in lines:
        line = json.loads(line)
        print(json.dumps(line, indent=2, ensure_ascii=False))

with open("data/converted_data/dj_back_to_llava.json", "r") as f:
    dj_back_to_llava = json.load(f)
    print("dj_back_to_llava:")
    print(json.dumps(dj_back_to_llava, indent=2, ensure_ascii=False))

Converting LLaVA format to Data-Juicer format...
2025-10-16 09:56:52.510 | INFO     | __main__:main:163 - Loading original LLaVA dataset.
2025-10-16 09:56:52.510 | INFO     | __main__:main:165 - Load [2] samples.
100%|██████████████████████████████████████████| 2/2 [00:00<00:00, 50533.78it/s]
2025-10-16 09:56:52.511 | INFO     | __main__:main:289 - Store the target dataset into [data/converted_data/llava_to_dj.jsonl].

Converting Data-Juicer format back to LLaVA format...
2025-10-16 09:56:53.327 | INFO     | __main__:main:148 - Start to convert.
2it [00:00, 26051.58it/s]
2025-10-16 09:56:53.328 | INFO     | __main__:main:236 - Start to write the converted dataset to [data/converted_data/dj_back_to_llava.json]...
Looking at the converted Data-Juicer format:
llava_to_dj:
{
  "id": "sample_1",
  "text": "[[human]]: <image>\nWhat do you see in this image?\n[[gpt]]: I can see a comfortable bed. <|__dj__eoc|>",
  "images": [
    "../../../tests/ops/data/img1.png"
  ]
}
{
  "id": "sample_2",


## Multi-modal Processing Workflow

Now let's put it all together with a workflow that demonstrates the power of Data-Juicer's unified approach to multi-modal data processing.

In [5]:
# Create a simplified image processing configuration using the llava_to_dj.jsonl
image_config = """
# Image processing demo config
project_name: 'image-processing-demo'
dataset_path: './data/converted_data/llava_to_dj.jsonl'
export_path: './outputs/demo/processed-images.jsonl'
np: 1

# Multi-modal settings
image_key: 'images'
image_special_token: '<__dj__image>'

process:
  # Filter images based on size
  - image_size_filter:
      min_size: "1KB"
      max_size: "10MB"
      any_or_all: any
  
  # Filter images based on aspect ratio
  - image_aspect_ratio_filter:
      min_ratio: 0.5
      max_ratio: 2.0
      any_or_all: any
"""

# Write the simplified config
with open('configs/image_processing.yaml', 'w') as f:
    f.write(image_config)

print("Image processing config created at configs/image_processing.yaml")

Image processing config created at configs/image_processing.yaml


In [6]:
# Run the actual processing command
print("\n🚀 Running multi-modal processing pipeline...")
!dj-process --config configs/image_processing.yaml


🚀 Running multi-modal processing pipeline...
2025-10-16 09:56:55.872 | INFO     | data_juicer.ops:timing_context:12 - Importing operator modules took 2.01 seconds
2025-10-16 09:56:56.692 | INFO     | data_juicer.config.config:660 - dataset_path config is set and a valid local path
2025-10-16 09:56:56.709 | INFO     | data_juicer.config.config:970 - Back up the input config file [/home/cmgzn/data-juicer/notebook/configs/image_processing.yaml] into the work_dir [/home/cmgzn/data-juicer/notebook/outputs/demo]
2025-10-16 09:56:56.712 | INFO     | data_juicer.config.config:991 - Configuration table: 
╒══════════════════════════╤════════════════════════════════════════════════════════════════════════════════════════════════╕
│ key                      │ values                                                                                         │
╞══════════════════════════╪════════════════════════════════════════════════════════════════════════════════════════════════╡
│ config          

## Data Path Management and Migration

After being processed by Data-Juicer, the output multimodal data paths are absolute and could originate from diverse sources, being stored under various root directories. 

To convert absolute paths to relative paths and facilitate data migration, we provide a tool to carry out this transformation. This tool also supports copying related multimodal data from multiple sources into a single directory. Both input and output of this utility conform to Data-Juicer’s data format. 

In [7]:
# Check the results obtained in the previous section.
abs_dir = None
with open("outputs/demo/processed-images.jsonl", "r") as f:
    lines = f.readlines()
    for item in lines:
        sample = json.loads(item)
        print(sample)
        image_path = sample.get('images', [])[0]
        # Get absolute directory path for subsequent path conversion operations
        abs_dir = os.path.dirname(image_path)

{'id': 'sample_1', 'text': '[[human]]: <image>\nWhat do you see in this image?\n[[gpt]]: I can see a comfortable bed. <|__dj__eoc|>', 'images': ['/home/cmgzn/data-juicer/notebook/data/converted_data/../../../tests/ops/data/img1.png']}
{'id': 'sample_2', 'text': '[[human]]: <image>\nDescribe the weather in this photo.\n[[gpt]]: The weather in the photo is distinctly rainy. <|__dj__eoc|>', 'images': ['/home/cmgzn/data-juicer/notebook/data/converted_data/../../../tests/ops/data/img3.jpg']}


Let's demonstrate the usage of the path management tool.

To learn more about the usage of the absolute to relative path conversion tool, you can execute the following command:

```bash
python tools/fmt_conversion/multimodal/absolute_path_to_relative_path.py --help
```

In [ ]:
!python ../tools/fmt_conversion/multimodal/absolute_path_to_relative_path.py\
    --dj_ds_path outputs/demo/processed-images.jsonl \
    --absolute_dir {abs_dir} \
    --path_key images \
    --target_dj_ds_path data/converted_data/relative_paths.jsonl \
    --target_mt_dir data/unified_media

2025-10-16 09:57:11.830 | INFO     | __main__:convert_absolute_path_to_relative_path:115 - Start to convert absolute path to relative path.
2it [00:00, 3004.52it/s]
2025-10-16 09:57:11.832 | INFO     | __main__:convert_absolute_path_to_relative_path:152 - Start to write the converted dataset to [data/converted_data/relative_paths.jsonl]...


In [9]:
with open("data/converted_data/relative_paths.jsonl", "r") as f:
    lines = f.readlines()
    for item in lines:
        print(json.loads(item))

{'id': 'sample_1', 'text': '[[human]]: <image>\nWhat do you see in this image?\n[[gpt]]: I can see a comfortable bed. <|__dj__eoc|>', 'images': ['img1.png'], '__dj__meta__': {'abs_dir': {'images': ['/home/cmgzn/data-juicer/notebook/data/converted_data/../../../tests/ops/data/']}}}
{'id': 'sample_2', 'text': '[[human]]: <image>\nDescribe the weather in this photo.\n[[gpt]]: The weather in the photo is distinctly rainy. <|__dj__eoc|>', 'images': ['img3.jpg'], '__dj__meta__': {'abs_dir': {'images': ['/home/cmgzn/data-juicer/notebook/data/converted_data/../../../tests/ops/data/']}}}


In [10]:
# Clean up temporary files
# !rm -rf configs/
# !rm -rf data/
# !rm -rf outputs/

## Next Steps

Continue with the next notebook to learn about distributed processing with Data-Juicer, which is essential for handling large-scale multi-modal datasets.